In [1]:
import pandas as pd
import numpy as np
import cobra
from CBModellingFuncs import *
import time
import os
import threadpoolctl

processes = 50
threadpoolctl.threadpool_limits(limits=processes)
os.makedirs("../results/sampling/thinning_test", exist_ok=True)
cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"

In [2]:
model, m_warnings = cobra.io.validate_sbml_model("../results/AdaptediMT1026v3.xml")

model.solver = 'cplex'
model.tolerance = 1e-6


rgdict = {}
for g in model.groups:
    for r in g.members:
        
        rgdict[r.id] = g.id.title()

SBML errors in validation, check error log for details.
COBRA errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


In [3]:
DRinfo = pd.read_csv(f"../results/dataframes/DerivedRetentInfo.csv", index_col=0, header=[0,1])
IRinfo = pd.read_csv(f"../results/dataframes/InterpolatedRetentInfo.csv", index_col=0, header=[0,1])
FRinfo = pd.read_csv(f"../results/dataframes/FittedRetentInfo.csv", index_col=0, header=[0,1])

NGAMseries = pd.read_csv('../results/dataframes/NGAMseries.csv',index_col=0,squeeze=True)

model_viab_cols = ["Growth Rate model 1/h", "qS model mmol/gh","qP model g/gh", "qCO2 viable model mmol/gh", "qO2 viable model mmol/gh", "qStorGlyc mmol/gh"]
viab_cols = ["Average Growth Rate 1/h", "qS viable mmol/gh","qP g/gh", "qCO2 viable mmol/gh", "qO2 viable mmol/gh", "qStorGlyc mmol/gh"] 

## Sampling

In [4]:
co2=True
o2=True

In [7]:
%%time
run_time = []
sampling_results = []

equation_names = ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]
datasets = [DRinfo, DRinfo, IRinfo, FRinfo, DRinfo]

for tfactor in [1,10,100, 1000]:
    for i, equation in enumerate(equation_names):
        data = datasets[i].loc[:, model_viab_cols].copy()
        if equation in ["Consensus", "ScaledConsensus"]:
            data["qStorGlyc mmol/gh"] = 0
        
        time_0 = data.iloc[0,:]
        name = time_0.name
        values = time_0
        mu = values[0]
        sd = values[1]
    
        loop_start = time.time()


        with model as temp_model:
            
            set_biomass_objective(temp_model, equation, name)

            temp_model.reactions.ATPM.bounds = (NGAMseries[f"{equation}"],1000)
           
            solution = constrain_predict(temp_model, values, co2=co2)
           
            pred_mu = solution.objective_value
            
            if equation in ["Consensus", "ScaledConsensus"]:
                temp_model.reactions.get_by_id(equation).bounds = (pred_mu*0.95, pred_mu)
                
            else:
                temp_model.reactions.get_by_id(f"{equation}Biomass{name}").bounds = (pred_mu*0.95, pred_mu)

            flux_span = constrain_predict(temp_model, values, co2=co2, fluxva=True, processes=processes,fopt=0)
            remove_blocked(temp_model, flux_span, solution)


            chains = constrain_predict(temp_model, values, sampling=True, chains=4, thinning=tfactor, 
                                       loopless=True, processes=processes, co2=co2, n_samples=1250)

                 # ensure all have same number of samples, may have
            chains = [df.iloc[:min([df.shape[0] for df in chains]), :] for df in chains]
            print(f"{equation} chains with thinning factor of  {tfactor} all have length of {chains[0].shape[0]}") 

            for j, chain in enumerate(chains):
                chain["Chain"] = j+1
            chains = pd.concat(chains, ignore_index=True)
            chains.to_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz", 
                                compression={'method': 'gzip', 'compresslevel': 6, 'mtime': 1})
            
        loop_end = time.time()
        run_time.append((equation, tfactor, (loop_end - loop_start)/60))
                         
run_time = pd.DataFrame(run_time, columns=["Equation", "Thinning", "Time"])
run_time.to_csv("../results/dataframes/RunTime.csv")

2483 reactions before making consistent, 1447 after
Consensus chains with thinning factor of  1 all have length of 1250
2483 reactions before making consistent, 1449 after
ScaledConsensus chains with thinning factor of  1 all have length of 1250
2483 reactions before making consistent, 1442 after
ScaledDerived chains with thinning factor of  1 all have length of 1250
2483 reactions before making consistent, 1447 after
ScaledInterpolated chains with thinning factor of  1 all have length of 1250
2483 reactions before making consistent, 1442 after
ScaledFitted chains with thinning factor of  1 all have length of 1250
2483 reactions before making consistent, 1450 after
Consensus chains with thinning factor of  10 all have length of 1250
2483 reactions before making consistent, 1452 after
ScaledConsensus chains with thinning factor of  10 all have length of 1250
2483 reactions before making consistent, 1439 after
ScaledDerived chains with thinning factor of  10 all have length of 1250
2483 

In [87]:
%%time

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

all_stats = pd.DataFrame()
n_chains = 4

for tfactor in [1,10,100, 1000]:#, 10000]:
    
    con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestConsensus{name}T{tfactor}.pkl.gz")
    con_chains = [con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    scaled_con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestScaledConsensus{name}T{tfactor}.pkl.gz")
    scaled_con_chains = [scaled_con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    der_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestDerived{name}T{tfactor}.pkl.gz")
    der_chains = [der_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
   
    int_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestInterpolated{name}T{tfactor}.pkl.gz")
    int_chains = [int_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    fit_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestFitted{name}T{tfactor}.pkl.gz")
    fit_chains = [fit_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    shared_columns = list(set(con_chains[0].columns).intersection(scaled_con_chains[0].columns, 
                                                                  der_chains[0].columns, 
                                                                  int_chains[0].columns, 
                                                                  fit_chains[0].columns))
    
    con_chains = [df.loc[:,shared_columns] for df in con_chains]
    scaled_con_chains = [df.loc[:,shared_columns] for df in scaled_con_chains]
    der_chains = [df.loc[:,shared_columns] for df in der_chains]
    int_chains = [df.loc[:,shared_columns] for df in int_chains]
    fit_chains = [df.loc[:,shared_columns] for df in fit_chains]
 
    all_df = []

    for tup in [("Consensus", con_chains),
                ("ScaledConsensus", scaled_con_chains),
                ("Derived", der_chains),
                ("Interpolated", int_chains),
                ("Fitted", fit_chains)]:
    
        biocomp, model_chains = tup
        grouped, rxns = extractchains(model_chains)
        
        for_pool = list(zip(grouped,rxns))
      
        pool = Pool(processes=processes)
        all_series = pool.starmap(calculateDiagnostics, for_pool)
        pool.close()
        df = pd.concat(all_series,axis=1).T
       
        df["Biomass Composition"] = biocomp
        all_df.append(df)
        
    all_df = pd.concat(all_df, ignore_index=True)
    all_df["Thinning Factor"] = tfactor
    
    all_stats = pd.concat([all_stats, all_df], ignore_index=True)
        
all_stats.set_index(["Thinning Factor", "Biomass Composition"], inplace=True)
all_stats["Rhat Fails"] = all_stats["Rhat"] > 1.01
all_stats["ESS-Bulk Fails"] = all_stats["ESS-Bulk"] < 400
n_grouped_chains = all_stats.groupby(["Thinning Factor", "Biomass Composition"]).count()["Rxn"]


# summary_stats = all_stats[["Geweke Fails","ESS Split Fails", "Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = all_stats[["Geweke Fails","Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = (summary_stats > 0).groupby(["Thinning Factor", "Biomass Composition"]).sum()
temp_df = (summary_stats.divide(n_grouped_chains, axis=0) * 100)
# temp_df.columns=["% Geweke Fails","% seperated-ESS Fails","% Rhat Fails","% bulk-ESS Fails"]
temp_df.columns=["% Geweke Fails","% Rhat Fails","% bulk-ESS Fails"]
summary_stats = pd.concat([summary_stats,temp_df],axis=1)
summary_stats.to_csv(f"../results/dataframes/ThinningDiagnosticsSummary.csv")

CPU times: user 14 s, sys: 19.5 s, total: 33.5 s
Wall time: 43 s


In [88]:
summary_stats

Geweke Fails  Rhat Fails  ESS-Bulk Fails  \
Thinning Factor Biomass Composition                                             
1               Consensus                     828         460             745   
                Derived                       879         313             711   
                Fitted                        826         641             727   
                Interpolated                  842         369             707   
                ScaledConsensus               783         676             709   
10              Consensus                     894         311             492   
                Derived                       840         559             615   
                Fitted                        883         263             596   
                Interpolated                  864         632             620   
                ScaledConsensus               886         400             567   
100             Consensus                     438          29              22   
                Derived                       533          41              20   
                Fitted                        517          76              20   
                Interpolated                  502          24               4   
                ScaledConsensus               844          54              26   
1000            Consensus                     332          10               0   
                Derived                       368           2               2   
                Fitted                        455          15               0   
                Interpolated                  373           2               0   
                ScaledConsensus               447           0               4   

                                     % Geweke Fails  % Rhat Fails  \
Thinning Factor Biomass Composition                                 
1               Consensus                 93.559322     51.977401   
                Derived                   97.993311     34.894091   
                Fitted                    94.077449     73.006834   
                Interpolated              95.248869     41.742081   
                ScaledConsensus           95.838433     82.741738   
10              Consensus                 92.738589     32.261411   
                Derived                   90.032154     59.914255   
                Fitted                    91.597510     27.282158   
                Interpolated              93.506494     68.398268   
                ScaledConsensus           93.459916     42.194093   
100             Consensus                 46.895075      3.104925   
                Derived                   59.354120      4.565702   
                Fitted                    55.831533      8.207343   
                Interpolated              51.540041      2.464066   
                ScaledConsensus           83.896620      5.367793   
1000            Consensus                 35.853132      1.079914   
                Derived                   39.107333      0.212540   
                Fitted                    48.872180      1.611171   
                Interpolated              39.554613      0.212089   
                ScaledConsensus           48.376623      0.000000   

                                     % bulk-ESS Fails  
Thinning Factor Biomass Composition                    
1               Consensus                   84.180791  
                Derived                     79.264214  
                Fitted                      82.801822  
                Interpolated                79.977376  
                ScaledConsensus             86.780906  
10              Consensus                   51.037344  
                Derived                     65.916399  
                Fitted                      61.825726  
                Interpolated                67.099567  
                ScaledConsensus             59.810127  
100             Consensus                    2.355460  
       

In [89]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
roi = ["G6PDH2","PGI","PYK","MDHm","CSm","AKGMALtm","NADH2_u6cm","NADH2_u6mh","CYOR_u6m","CYOOm","ATPS3m","ATPM"]
failures[failures["Rxn"].isin(roi)]

Rxn Geweke Fails      Rhat  \
Thinning Factor Biomass Composition                                      
1               Consensus              CYOR_u6m            4  1.021581   
                Consensus                   CSm            1  1.055778   
                Consensus              AKGMALtm            4  1.043052   
                Consensus            NADH2_u6cm            4  1.031601   
                Consensus            NADH2_u6mh            4  1.035989   
...                                         ...          ...       ...   
10              Fitted                     MDHm            4  1.023809   
                Fitted                      PGI            2  1.009472   
                Fitted                    CYOOm            2   1.02049   
100             Consensus            NADH2_u6mh            2  1.010969   
                Consensus                ATPS3m            2  1.010974   

                                    ESS Split Fails     ESS-Bulk     ESS-Tail  \
Thinning Factor Biomass Composition                                             
1               Consensus                         0   210.924266   242.117459   
                Consensus                         0   140.745101    90.707419   
                Consensus                         0   227.631605   221.576272   
                Consensus                         0    170.42466   215.500467   
                Consensus                         0   201.036961   221.681081   
...                                             ...          ...          ...   
10              Fitted                            0   303.928596   312.455698   
                Fitted                            0   315.077315   289.436529   
                Fitted                            0     292.0375   278.861082   
100             Consensus                         0  1389.715576  1724.465225   
                Consensus                         0  1295.283529  1595.595974   

                                     Rhat Fails  ESS-Bulk Fails  
Thinning Factor Biomass Composition                              
1               Consensus                  True            True  
                Consensus                  True            True  
                Consensus                  True            True  
                Consensus                  True            True  
                Consensus                  True            True  
...                                         ...             ...  
10              Fitted                     True            True  
                Fitted                    False            True  
                Fitted                     True            True  
100             Consensus                  True           False  
                Consensus                  True           False  

[112 rows x 8 columns]

In [90]:
failures = all_stats[all_stats.loc[:,["Geweke Fails","Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()

failures.loc[:,"Subsystem"] = failures["Rxn"].apply(lambda x:rgdict.get(x))

In [91]:
test = failures[(failures["Rhat Fails"] > 0)^(failures["ESS-Bulk Fails"] > 0)]
sum_fails_by_subsystem = test.groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()[["Rhat Fails", "ESS-Bulk Fails"]]
sum_fails_by_subsystem = sum_fails_by_subsystem.groupby(["Thinning Factor", "Subsystem"]).sum().unstack([0])
sum_fails_by_subsystem = sum_fails_by_subsystem.fillna(0)
sum_fails_by_subsystem

Rhat Fails              \
Thinning Factor                                          1     10    100    
Subsystem                                                                   
Alanine And Aspartate Metabolism                          0.0   3.0   4.0   
Amino Sugar And Nucleotide Sugar Metabolism               0.0   1.0   0.0   
Arginine And Proline Metabolism                           1.0  11.0   7.0   
Biomass Composition                                       0.0   0.0   0.0   
Carbohydrate Metabolism                                   0.0   0.0   0.0   
Citric Acid Cycle                                         2.0   3.0   3.0   
Complex Alcohol Metabolism                                0.0   0.0   0.0   
Cysteine Metabolism                                       0.0   0.0   2.0   
Exchange Reaction                                         0.0   3.0   0.0   
Fatty Acid Biosynthesis                                   7.0   5.0  14.0   
Fatty Acid Degradation                                    1.0  56.0  20.0   
Folate Metabolism                                         2.0   1.0   6.0   
Fructose And Mannose Metabolism                           0.0   0.0   0.0   
Galactose Metabolism                                      0.0   0.0   0.0   
Glutamate Metabolism                                      4.0   4.0   0.0   
Glutamine Metabolism                                      0.0   0.0   0.0   
Glycerolipid Metabolism                                   2.0   2.0   5.0   
Glycine And Serine Metabolism                             0.0   1.0   4.0   
Glycolysis/Gluconeogenesis                                0.0   1.0   0.0   
Glycosylation Pathway                                     0.0   0.0   0.0   
Glyoxylate And Dicarboxylate Metabolism                   3.0   0.0   2.0   
Histidine Metabolism                                      0.0   0.0   6.0   
Inositol Phosphate Metabolism                             0.0   0.0   0.0   
Methanol Metabolism                                       0.0   0.0   2.0   
Methionine Metabolism                                     1.0   3.0   5.0   
N-Glycan Metabolism                                       0.0   0.0   0.0   
Nad Biosynthesis                                          3.0   4.0   0.0   
Nucleotide Metabolism                                    10.0  13.0  16.0   
Other                                                     0.0   0.0   0.0   
Other Amino Acid Metabolism                               3.0   2.0   6.0   
Oxidative Phosphorylation                                 2.0   3.0   2.0   
Pantothenate And Coa Biosynthesis                         1.0   0.0   4.0   
Pentose Phosphate Pathway                                 0.0   1.0   0.0   
Phospholipid Metabolism                                   0.0   7.0   8.0   
Porphyrin And Chlorophyll Metabolism                      0.0   1.0   0.0   
Pyridoxine Metabolism                                    13.0   9.0   1.0   
Pyruvate Metabolism                                       5.0   9.0   4.0   
Quinone Biosynthesis                                      0.0   0.0   2.0   
Recombinant Protein Production                            0.0   0.0   0.0   
Riboflavin Metabolism                                     0.0   7.0   5.0   
Sphingolipid Metabolism                                   6.0  30.0   3.0   
Sterol Metabolism                                         0.0   4.0   1.0   
Thiamine Metabolism                                       3.0  13.0   0.0   
Threonine And Lysine Metabolism                           0.0   8.0   0.0   
Transport, Er                                             0.0   1.0   5.0   
Transport, Extracellular                                  0.0   4.0   0.0   
Transport, Golgi Apparatus                                0.0   4.0   0.0   
Transport, Mitochondrial                                  4.0  10.0  11.0   
Transport, Nuclear                                        0.0   4.0   0.0   
Transport, Peroxisomal                         

In [92]:
rhat_fails = failures[failures["Rhat Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
rhat_fails = rhat_fails.groupby(["Thinning Factor", "Biomass Composition"])["Rhat Fails"].nlargest(5)
rhat_fails.index = rhat_fails.index.droplevel([0,1])
rhat_fails = rhat_fails.to_frame()
rhat_fails

Rhat Fails
Thinning Factor Biomass Composition Subsystem                                             
1               Consensus           Fatty Acid Biosynthesis                             60
                                    Transport, Mitochondrial                            42
                                    Nucleotide Metabolism                               26
                                    Transport, Peroxisomal                              20
                                    Valine, Leucine, And Isoleucine Metabolism          19
...                                                                                    ...
1000            Fitted              Riboflavin Metabolism                                2
                                    Sphingolipid Metabolism                              2
                                    Glycerolipid Metabolism                              1
                Interpolated        Exchange Reaction                                    1
                                    Transport, Extracellular                             1

[88 rows x 1 columns]

In [93]:
essbulk_fails = failures[failures["ESS-Bulk Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
if not essbulk_fails.empty:
    sum_essbulk = essbulk_fails.groupby(["Thinning Factor", "Biomass Composition"])["ESS-Bulk Fails"].nlargest(5)
    sum_essbulk.index = sum_essbulk.index.droplevel([0,1])
    sum_essbulk = sum_essbulk.to_frame()
    print(sum_essbulk)
else:
    print("No ESS Bulk fails")

                                                              ESS-Bulk Fails
Thinning Factor Biomass Composition Subsystem                               
1               Consensus           Fatty Acid Biosynthesis               77
                                    Transport, Mitochondrial              58
                                    Sphingolipid Metabolism               56
                                    Fatty Acid Degradation                54
                                    Nucleotide Metabolism                 47
...                                                                      ...
1000            Derived             Exchange Reaction                      1
                                    Transport, Extracellular               1
                ScaledConsensus     Nad Biosynthesis                       2
                                    Exchange Reaction                      1
                                    Transport, Extracellular               1

In [94]:
geweke_fails = failures[failures["Geweke Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Rxn"]).count()
geweke_fails = geweke_fails.groupby(["Thinning Factor", "Biomass Composition"])["Geweke Fails"].nlargest(5)
geweke_fails.index = geweke_fails.index.droplevel([0,1])
geweke_fails = geweke_fails.to_frame()
geweke_fails

Geweke Fails
Thinning Factor Biomass Composition Rxn                   
1               Consensus           10FTHFtm             1
                                    12DGRter             1
                                    13DAMPPO             1
                                    13GS                 1
                                    1PYR5Ctm             1
...                                                    ...
1000            ScaledConsensus     13DAMPPO             1
                                    2DDA7Ptm             1
                                    2DHPtm               1
                                    34HPLFM              1
                                    34HPPt2m             1

[100 rows x 1 columns]

### Additional diagnostic plots of a selection of reactions (used in later analysis)
Additional plots include Geweke plots, Rank plots, local, quantile and evolution ESS plots

In [ ]:
%%time

from matplotlib.backends.backend_pdf import PdfPages

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

n_chains = 4
interesting_reactions = ["G6PDH2","PGI","PC","PYK","MDHm", 
                         "CSm", "AKGMALtm","NADH2_u6cm","NADH2_u6mh",
                         "SUCD2_u6m","CYOR_u6m","CYOOm","ATPS3m"]

for rxn in interesting_reactions:

    with PdfPages(f'../results/plots/SamplingDiagnostics{rxn}.pdf') as pdf:

        for tfactor in [1,10,100, 1000]:
            
            
            con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestConsensus{name}T{tfactor}.pkl.gz")
            con_chains = [con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]

            scaled_con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestScaledConsensus{name}T{tfactor}.pkl.gz")
            scaled_con_chains = [scaled_con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]

            der_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestDerived{name}T{tfactor}.pkl.gz")
            der_chains = [der_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]

            int_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestInterpolated{name}T{tfactor}.pkl.gz")
            int_chains = [int_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]

            fit_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestFitted{name}T{tfactor}.pkl.gz")
            fit_chains = [fit_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]

            for tup in [("Consensus", con_chains),
                        ("ScaledConsensus", scaled_con_chains),
                        ("Derived", der_chains),
                        ("Interpolated", int_chains),
                        ("Fitted", fit_chains)]:
                biocomp, model_chains = tup
                grouped, rxns = extractchains(model_chains)
                
                try:
                    chains = grouped[rxns.index(rxn)]

                    fig = plt.figure(figsize=(13, 12), constrained_layout=True)
                    spec = fig.add_gridspec(4, 5, height_ratios=[1,1,1,1], hspace=0.1, bottom=0.1)

                    for i, chain in enumerate(chains):

                        ax = fig.add_subplot(spec[i,0])
                        gw = az.geweke(chain,0.01,0.5,100)
                        plotGeweke(gw,1,50,100, ax, f"{biocomp}-{tfactor}")

                        ax = fig.add_subplot(spec[i, 1])
                        plotrank(chain,ax)


                        axes = [fig.add_subplot(spec[i, j+2]) for j in np.arange(3)]
                        calcESS(chain, axes, f"{biocomp}-{tfactor}")
                        
                    pdf.savefig(fig)
                    plt.close()
                    
               
                except:
                    print(f"{rxn} not present in {biocomp}, {tfactor}")
